In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr
from PIL import Image
import numpy as np
import os


/Users/utkarshverma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        
        if img is not None:
            # Convert BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Resize image
            img = cv2.resize(img, img_size)
            # Normalize to [0, 1]
            img = img / 255.0
            # Append to list
            images.append(img)
    
    return images



In [7]:
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        
        if img is not None:
            # Convert BGR to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Resize image
            img = cv2.resize(img, img_size)
            # Normalize to [0, 1]
            img = img / 255.0
            # Append to list
            images.append(img)
    
    return images

# Adjust the folder paths for macOS
noisy_folder_path = '/Users/utkarshverma/vlg/low'
clean_folder_path = '/Users/utkarshverma/vlg/high'
img_size = (128, 128)  # Desired image size

noisy_images = load_images_from_folder(noisy_folder_path, img_size)
clean_images = load_images_from_folder(clean_folder_path, img_size)

print(f"Loaded {len(noisy_images)} noisy images.")
print(f"Loaded {len(clean_images)} clean images.")



Loaded 485 noisy images.
Loaded 485 clean images.


In [26]:
train_noisy, test_noisy, train_clean, test_clean = train_test_split(
    noisy_images, clean_images, test_size=0.2, random_state=42
)


In [27]:
def preprocess_image(image):
    return tf.image.convert_image_dtype(image, tf.float32)

# Create TensorFlow datasets
train_noisy_ds = tf.data.Dataset.from_tensor_slices(train_noisy).map(preprocess_image).batch(32)
train_clean_ds = tf.data.Dataset.from_tensor_slices(train_clean).map(preprocess_image).batch(32)
test_noisy_ds = tf.data.Dataset.from_tensor_slices(test_noisy).map(preprocess_image).batch(32)
test_clean_ds = tf.data.Dataset.from_tensor_slices(test_clean).map(preprocess_image).batch(32)

# Combine noisy and clean images for training
train_ds = tf.data.Dataset.zip((train_noisy_ds, train_clean_ds))
test_ds = tf.data.Dataset.zip((test_noisy_ds, test_clean_ds))


In [31]:
def create_denoising_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    return model

input_shape = (128, 128, 3)
model = create_denoising_model(input_shape)
model.compile(optimizer='adam', loss='mean_squared_error')



In [29]:
history = model.fit(train_ds, epochs=40, validation_data=test_ds)


13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 394ms/step - loss: 0.0482 - val_loss: 0.0393


In [30]:
predicted_images = model.predict(test_noisy_ds)

psnr_values = [psnr(test_clean[i], predicted_images[i]) for i in range(len(test_clean))]
average_psnr = np.mean(psnr_values)
print(f'Average PSNR: {average_psnr}')


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Average PSNR: 14.653218603099651


/var/folders/hv/br5r3n552kx3klh4xq4rsk100000gn/T/ipykernel_46122/69829370.py:3: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  psnr_values = [psnr(test_clean[i], predicted_images[i]) for i in range(len(test_clean))]
